일단 코드 쭉 연결시켜 보고 이해하기

In [20]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Dot
import tensorflow.keras.backend as K

# Attention.py 

```
# def _scaled_dot_product(qs, ks, vs):
#     key_dim_per_head = key_dim // num_heads
    
#     o1 = tf.matmul(qs, ks, transpose_b=True) # Q * (K^T)
#     o2 = o1 / (key_dim_per_head**0.5) # Q * (K^T)/(루트dk)
#     tf
#     if masked:
#         diag_vals = tf.ones_like(o2) # batch_size, num_heads 축 없애고 mask 2차원으로 만듦.
#         tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense() # 대각요소 위 무시.
#         masks = tf.tile(input=tril[np.newaxis, np.newaxis, :, :],
#                         multiples=[o2.shape[0], o2.shape[1], 1, 1]) # batch size, num_heads 만큼 mask 늘려 줌.
#         paddings = tf.ones_like(masks) * -1e9
#         o2 = tf.where(tf.equal(masks, 0), paddings, o2)
    
#     o3 = Activation('softmax')(o3)
#     return tf.matmul(o3, vs)
```

In [103]:
def _scaled_dot_product(qs, ks, vs, masked):
    '''
    qs, ks, vs: 4-D tensor (batch_size, num_heads, max_seq_len, dim)
    masked: boolean
    '''
    print(qs.shape, ks.shape)
    key_dim_per_head = linear_key_dim // num_heads

    o1 = tf.matmul(qs, ks, transpose_b=True) # Q *a (K^T)
    o2 = o1 / (key_dim_per_head**0.5) # Q * (K^T)/(루트dk)
    
    if masked:
        diag_vals = tf.ones_like(o2[0, 0, :, :])
        tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense() # 하삼각행렬: 자기 자신인 단어까지만.
        masks = tf.tile(input=tf.reshape(tril, (1, 1, tril.shape[0], tril.shape[1])),
                        multiples=[tf.shape(o2)[0], tf.shape(o2)[1], 1, 1]) # batch size, num_heads 만큼 행렬 사이즈 늘려서 마스크 만듦.
        paddings = tf.ones_like(masks) * -1e9 # 주목하지 말아야 할 부분에 낮은 숫자.
        o2 = tf.where(tf.equal(masks, 0), paddings, o2) # mask에 0인 위치(뒤의 단어)에 패딩을 넣어 준다.

    o3 = Activation('softmax')(o2) # softmax
    o3 = tf.matmul(o3, vs)

    return o3   

In [19]:
def _concat_heads(outputs):
    def transpose_then_concat_last_two_dimension(tensor):
        tensor = tf.transpose(tensor, [0, 2, 1, 3])
        t_shape = tensor.get_shape().as_list()
        num_heads, dim = t_shape[-2:]
        return tf.reshape(tensor, [-1] + t_shape[1:-2] + [num_heads * dim])
    return transpose_then_concat_last_two_dimension(outputs)

테스트 및 정리 요망
- Keras Dot 연산 쓰니까 이상했다.

In [8]:
# ones_like -> 차원 삭제
tensor = tf.constant(np.arange(1, 13).reshape(1, 2, 2, 3))
print(tensor)
print(tf.ones_like(tensor[0, 0, :, :])) # 앞의 두 차원 사라짐.

tf.Tensor(
[[[[ 1  2  3]
   [ 4  5  6]]

  [[ 7  8  9]
   [10 11 12]]]], shape=(1, 2, 2, 3), dtype=int64)
tf.Tensor(
[[1 1 1]
 [1 1 1]], shape=(2, 3), dtype=int64)


In [14]:
# linear operator shape 얻기
tensor = [[1., 2.], [3., 4.]]
tril = tf.linalg.LinearOperatorLowerTriangular(tensor).to_dense()
print(tril.shape)
print(tril.get_shape().as_list())  # 리스트로 해주려고...

print(tril)
# 위나 아래나 똑같다.
print(tril[np.newaxis, np.newaxis, :, :])
print(tf.reshape(tril,  [1, 1] + tril.get_shape().as_list()))

(2, 2)
[2, 2]
tf.Tensor(
[[1. 0.]
 [3. 4.]], shape=(2, 2), dtype=float32)
tf.Tensor(
[[[[1. 0.]
   [3. 4.]]]], shape=(1, 1, 2, 2), dtype=float32)
tf.Tensor(
[[[[1. 0.]
   [3. 4.]]]], shape=(1, 1, 2, 2), dtype=float32)


In [10]:
# input x multiple 차원만큼 shape된다.
a = tf.constant([[1, 2, 3], [4, 5, 6]], tf.int32) # (2, 3)
b = tf.constant([1, 2], tf.int32) # (2*1, 3*2) = (2, 6)
c = tf.constant([2, 1], tf.int32) # (2*2, 3*1) = (4, 3)
d = tf.constant([2, 2], tf.int32) # (2*2, 3*2) = (4, 6)
e = tf.constant([1, 2, 2], tf.int32) # 오류나지 않을까? 
print(tf.tile(a, b))
print(tf.tile(a, c))
print(tf.tile(a, d))
print(tf.tile(a, e)) # 오케 오류 났다.

tf.Tensor(
[[1 2 3 1 2 3]
 [4 5 6 4 5 6]], shape=(2, 6), dtype=int32)
tf.Tensor(
[[1 2 3]
 [4 5 6]
 [1 2 3]
 [4 5 6]], shape=(4, 3), dtype=int32)
tf.Tensor(
[[1 2 3 1 2 3]
 [4 5 6 4 5 6]
 [1 2 3 1 2 3]
 [4 5 6 4 5 6]], shape=(4, 6), dtype=int32)


InvalidArgumentError: ignored

In [32]:
from tensorflow.keras.layers import Multiply, multiply
dk = 2
Q_Kt = tf.constant([[15.402, 14.309, 12.622, 8.961, 7.433],
                    [12.875, 11.969, 10.502, 7.402, 6.120],
                    [11.866, 11.002, 9.856, 7.145, 5.976],
                    [9.482, 8.778, 7.954, 5.852, 4.924],
                    [9.642, 8.926, 8.096, 5.965, 5.021]],
                    dtype=tf.float32)
Q_Kt / dk**(0.5)

<tf.Tensor: shape=(5, 5), dtype=float32, numpy=
array([[10.89086  , 10.117991 ,  8.925102 ,  6.3363843,  5.2559247],
       [ 9.104    ,  8.463361 ,  7.4260354,  5.2340045,  4.3274937],
       [ 8.39053  ,  7.7795887,  6.9692445,  5.052278 ,  4.22567  ],
       [ 6.704787 ,  6.2069836,  5.6243277,  4.137989 ,  3.4817936],
       [ 6.817924 ,  6.311635 ,  5.724736 ,  4.217892 ,  3.550383 ]],
      dtype=float32)>

In [46]:
q = tf.constant([[4.134, 3.328],
                 [3.567, 2.635],
                 [2.885, 2.959],
                 [2.129, 2.598],
                 [2.147, 2.666]], dtype=tf.float32)
k = tf.constant([[2.313, 1.754],
                 [2.178, 1.584],
                 [1.729, 1.645],
                 [1.041, 1.399],
                 [0.800, 1.239]])

q[np.newaxis, :, :], k[np.newaxis, :, :].shape # dummy batch_shape
Dot(axes=[2, 2])([q[np.newaxis, :, :], k[np.newaxis, :, :]]) # 이렇게 나와야 한다.

<tf.Tensor: shape=(1, 5, 5), dtype=float32, numpy=
array([[[15.399253 , 14.275404 , 12.622246 ,  8.959366 ,  7.430592 ],
        [12.872261 , 11.942766 , 10.501917 ,  7.399612 ,  6.118365 ],
        [11.8630905, 10.970586 ,  9.8557205,  7.142926 ,  5.974201 ],
        [ 9.481269 ,  8.752193 ,  7.954751 ,  5.850891 ,  4.922122 ],
        [ 9.642175 ,  8.89911  ,  8.097733 ,  5.964761 ,  5.020774 ]]],
      dtype=float32)>

In [82]:
q = tf.constant([[4.134, 3.328],
                 [3.567, 2.635],
                 [2.885, 2.959],
                 [2.129, 2.598],
                 [2.147, 2.666]], dtype=tf.float32)
k = tf.constant([[2.313, 1.754],
                 [2.178, 1.584],
                 [1.729, 1.645],
                 [1.041, 1.399],
                 [0.800, 1.239]], dtype=tf.float32)
v = tf.constant([[3.382, 4.593],
                 [3.006, 3.579],
                 [3.258, 4.015],
                 [2.749, 3.627],
                 [2.609, 3.605]], dtype=tf.float32)

q = q[np.newaxis, np.newaxis, :, :]
k = k[np.newaxis, np.newaxis, :, :]
v = v[np.newaxis, np.newaxis, :, :]
print(q.shape, k.shape, v.shape)
_scaled_dot_product(q, k, v)

(1, 1, 5, 2) (1, 1, 5, 2) (1, 1, 5, 2)
(1, 1, 5, 2) (1, 1, 5, 2)


<tf.Tensor: shape=(1, 1, 5, 2), dtype=float32, numpy=
array([[[[3.2595491, 4.248784 ],
         [3.244916 , 4.2112794],
         [3.2370493, 4.193578 ],
         [3.213495 , 4.147224 ],
         [3.2150795, 4.1502357]]]], dtype=float32)>

In [104]:
def _scaled_dot_product(qs, ks, vs, masked=True):
    '''
    qs: (batch_size, num_heads, max_seq_len, dim)
    ks: (batch_size, num_heads, max_seq_len, dim)
    vs: (batch_size, num_heads, max_seq_len, dim)
    '''
    print(qs.shape, ks.shape, vs.shape)
    # key_dim_per_head = linear_key_dim // num_heads
    key_dim_per_head = 2 # 테스트용

    o1 = tf.matmul(qs, ks, transpose_b=True) # Q *a (K^T)
    o2 = o1 / (key_dim_per_head**0.5) # Q * (K^T)/(루트dk)

    print("----- o2 -----")
    print(o2.shape)
    if masked:
        diag_vals = tf.ones_like(o2[0, 0, :, :])
        print(diag_vals)
        tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense() # 자기 자신까지만
        print(tril.shape)
        masks = tf.tile(input=tf.reshape(tril, (1, 1, tril.shape[0], tril.shape[1])),
                        multiples=[tf.shape(o2).sh[0], o2.shape[1], 1, 1]) # batch size, num_heads 만큼 mask 늘려 줌.
        print(f"===== masks =======")
        print(masks)
        print()
        paddings = tf.ones_like(masks) * -1e9 # 주목하지 말아야 할 부분에 낮은 숫자 주고
        print(f"===== paddings ====")
        print(paddings)
        print()
        o2 = tf.where(tf.equal(masks, 0), paddings, o2) # mask에 0인 위치(뒤의 단어)에 패딩을 넣어 준다.
        print(o2)

    o3 = Activation('softmax')(o2) # softmax
    o3 = tf.matmul(o3, vs)

    return o3

_scaled_dot_product(q, k, v)

(1, 1, 5, 2) (1, 1, 5, 2) (1, 1, 5, 2)
----- o2 -----
(1, 1, 5, 5)
tf.Tensor(
[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]], shape=(5, 5), dtype=float32)
(5, 5)
===== masks =======
tf.Tensor(
[[[[1. 0. 0. 0. 0.]
   [1. 1. 0. 0. 0.]
   [1. 1. 1. 0. 0.]
   [1. 1. 1. 1. 0.]
   [1. 1. 1. 1. 1.]]]], shape=(1, 1, 5, 5), dtype=float32)

===== paddings ====
tf.Tensor(
[[[[-1.e+09 -1.e+09 -1.e+09 -1.e+09 -1.e+09]
   [-1.e+09 -1.e+09 -1.e+09 -1.e+09 -1.e+09]
   [-1.e+09 -1.e+09 -1.e+09 -1.e+09 -1.e+09]
   [-1.e+09 -1.e+09 -1.e+09 -1.e+09 -1.e+09]
   [-1.e+09 -1.e+09 -1.e+09 -1.e+09 -1.e+09]]]], shape=(1, 1, 5, 5), dtype=float32)

tf.Tensor(
[[[[ 1.0888916e+01 -1.0000000e+09 -1.0000000e+09 -1.0000000e+09
    -1.0000000e+09]
   [ 9.1020632e+00  8.4448109e+00 -1.0000000e+09 -1.0000000e+09
    -1.0000000e+09]
   [ 8.3884716e+00  7.7573757e+00  6.9690471e+00 -1.0000000e+09
    -1.0000000e+09]
   [ 6.7042694e+00  6.1887355e+00  5.6248584e+00  4.1372046e+00


<tf.Tensor: shape=(1, 1, 5, 2), dtype=float32, numpy=
array([[[[3.382    , 4.593    ],
         [3.25365  , 4.2468643],
         [3.2523253, 4.2100797],
         [3.2254434, 4.1579413],
         [3.2150795, 4.1502357]]]], dtype=float32)>

In [84]:
q # batch_size: 1, num_heads: 1, max_seq :5, dim = 2

<tf.Tensor: shape=(1, 1, 5, 2), dtype=float32, numpy=
array([[[[4.134, 3.328],
         [3.567, 2.635],
         [2.885, 2.959],
         [2.129, 2.598],
         [2.147, 2.666]]]], dtype=float32)>

In [85]:
tf.reshape(q, shape=(-1, q.shape[2], q.shape[1]*q.shape[3]))

TensorShape([1, 1, 5, 2])

In [88]:
def _concat_heads(outputs):
    def transpose_then_concat_last_two_dimension(tensor):
        return tf.reshape(tensor, shape=(-1, tensor.shape[2], tensor.shape[1]*tensor.shape[3]))
    return transpose_then_concat_last_two_dimension(outputs)

In [89]:
_concat_heads(q)

<tf.Tensor: shape=(1, 5, 2), dtype=float32, numpy=
array([[[4.134, 3.328],
        [3.567, 2.635],
        [2.885, 2.959],
        [2.129, 2.598],
        [2.147, 2.666]]], dtype=float32)>

o2 shape 주면 안된다.

In [ ]:
TypeError                                 Traceback (most recent call last)
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/tensor_util.py in make_tensor_proto(values, dtype, shape, verify_shape, allow_broadcast)
    547     try:
--> 548       str_values = [compat.as_bytes(x) for x in proto_values]
    549     except TypeError:

11 frames
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/tensor_util.py in <listcomp>(.0)
    547     try:
--> 548       str_values = [compat.as_bytes(x) for x in proto_values]
    549     except TypeError:

/usr/local/lib/python3.6/dist-packages/tensorflow/python/util/compat.py in as_bytes(bytes_or_text, encoding)
     86     raise TypeError('Expected binary or unicode string, got %r' %
---> 87                     (bytes_or_text,))
     88 

TypeError: Expected binary or unicode string, got None

During handling of the above exception, another exception occurred:

TypeError                                 Traceback (most recent call last)
<ipython-input-14-da5338f21150> in <module>()
      5 
      6 
----> 7 decMaskedAttValue = _scaled_dot_product(decMaskedQs, decMaskedKs, decMaskedVs, key_dim, True)
      8 decMaskedConAttValue = _concat_heads(decMaskedAttValue)
      9 decMaskedConAttValue.shape

<ipython-input-9-21994e08574a> in _scaled_dot_product(qs, ks, vs, key_dim, masked)
     12         tril = tf.linalg.LinearOperatorLowerTriangular(diag_vals).to_dense() # 하삼각행렬: 자기 자신인 단어까지만.
     13         masks = tf.tile(input=tf.reshape(tril, (1, 1, tril.shape[0], tril.shape[1])),
---> 14                         multiples=[o2.shape[0], o2.shape[1], 1, 1]) # batch size, num_heads 만큼 행렬 사이즈 늘려서 마스크 만듦.
     15         paddings = tf.ones_like(masks) * -1e9 # 주목하지 말아야 할 부분에 낮은 숫자.
     16         o2 = tf.where(tf.equal(masks, 0), paddings, o2) # mask에 0인 위치(뒤의 단어)에 패딩을 넣어 준다.

/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gen_array_ops.py in tile(input, multiples, name)
  11404   try:
  11405     _, _, _op, _outputs = _op_def_library._apply_op_helper(
> 11406         "Tile", input=input, multiples=multiples, name=name)
  11407   except (TypeError, ValueError):
  11408     result = _dispatch.dispatch(

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py in _apply_op_helper(op_type_name, name, **keywords)
    471         except TypeError as err:
    472           if dtype is None:
--> 473             raise err
    474           else:
    475             raise TypeError(

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/op_def_library.py in _apply_op_helper(op_type_name, name, **keywords)
    468               dtype=dtype,
    469               as_ref=input_arg.is_ref,
--> 470               preferred_dtype=default_dtype)
    471         except TypeError as err:
    472           if dtype is None:

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/ops.py in convert_to_tensor(value, dtype, name, as_ref, preferred_dtype, dtype_hint, ctx, accepted_result_types)
   1497 
   1498     if ret is None:
-> 1499       ret = conversion_func(value, dtype=dtype, name=name, as_ref=as_ref)
   1500 
   1501     if ret is NotImplemented:

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/constant_op.py in _constant_tensor_conversion_function(v, dtype, name, as_ref)
    336                                          as_ref=False):
    337   _ = as_ref
--> 338   return constant(v, dtype=dtype, name=name)
    339 
    340 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/constant_op.py in constant(value, dtype, shape, name)
    262   """
    263   return _constant_impl(value, dtype, shape, name, verify_shape=False,
--> 264                         allow_broadcast=True)
    265 
    266 

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/constant_op.py in _constant_impl(value, dtype, shape, name, verify_shape, allow_broadcast)
    280       tensor_util.make_tensor_proto(
    281           value, dtype=dtype, shape=shape, verify_shape=verify_shape,
--> 282           allow_broadcast=allow_broadcast))
    283   dtype_value = attr_value_pb2.AttrValue(type=tensor_value.tensor.dtype)
    284   attrs = {"value": tensor_value, "dtype": dtype_value}

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/tensor_util.py in make_tensor_proto(values, dtype, shape, verify_shape, allow_broadcast)
    550       raise TypeError("Failed to convert object of type %s to Tensor. "
    551                       "Contents: %s. Consider casting elements to a "
--> 552                       "supported type." % (type(values), values))
    553     tensor_proto.string_val.extend(str_values)
    554     return tensor_proto

TypeError: Failed to convert object of type <class 'list'> to Tensor. Contents: [None, 4, 1, 1]. Consider casting elements to a supported type.